In [95]:
import tensorflow as tf
import tensorflow.keras as kr
import keras
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import math as mth
import random as random

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

from keras.optimizers import RMSprop

In [2]:
#DECLARACION DE CLASES
#Los vectores de 8 elementos se llaman "pixels" porque son un conjunto de elementos "pixel"
#Voy a generar un vector "dataset" que es un vector de 1000 elemenos X. Con Y vamos a entrenar la red.

class pixel:
    #esta es la clase pixel, contiene tres canales de entre 0 y 255 cada uno
    #por una cuestion de entendimiento los canales se van a llamar R,G y B pero no tiene nada que ver con los colores en si,
    #son solo numeros
    def __init__(self,tiempo,frecR = 1,frecG = 2, frecB = 5):
        a = mth.floor(random.uniform(0,10)) #Aqui esta la aleatoriedad necesaria para entrenar bien
        b = mth.floor(random.uniform(0,10))
        self.R = mth.floor((127 * mth.sin(frecR*tiempo + a)) + 128 + b) #Aqui genero los tres canales del pixel
        self.G = mth.floor((127 * mth.sin(frecG*tiempo + a)) + 128 + b)
        self.B = mth.floor((127 * mth.sin(frecB*tiempo + a)) + 128 + b)
        #Esto se hace para que los valores sigan dentro del rango 0-255 :
        if self.R > 255: 
            self.R = 255
        if self.G > 255: 
            self.G = 255
        if self.B > 255: 
            self.B = 255
        if self.R < 0: 
            self.R = 0
        if self.G < 0: 
            self.G = 0
        if self.B < 0: 
            self.B = 0
    def getData(self): #Esto es para visualizar rapido en caso de que sea necesario
        print("R:",self.R, "G:",self.G, "B:", self.B)
#   def getNumbers(self):
 #      R = self.R
        #G = self.G
        #B = self.B
        
class pixels(list): #Esta es la clase de 8 elementos (cada elemento es un pixel)
    def fillWithPixels(self, cantElementos = 8):
        for x in range(cantElementos):
            #print("Iteracion numero ", x)
            self.append(pixel(x, 0.2))

In [3]:
#prueba:
X = pixels()
X.fillWithPixels()
X[0].getData()
X[1].getData()
X[2].getData()
X[3].getData()
X[4].getData()
X[5].getData()
X[6].getData()
X[7].getData()

R: 36 G: 36 B: 36
R: 248 G: 147 B: 94
R: 219 G: 98 B: 65
R: 110 G: 215 B: 17
R: 221 G: 255 B: 245
R: 255 G: 12 B: 205
R: 165 G: 252 B: 174
R: 99 G: 95 B: 50


In [4]:
#GENERACION DEL DATASET DE ENTRENAMIENTO
#Quiero una matriz de 1000 filas. Cada fila es un vector de pixels. Es decir dataset1 y dataset2 son listas de 1000 elementos de pixels.
#Cada uno de ellos es un conjunto de 8 pixels

size = 10000
dataset1 = [] #Para entrenamiento
dataset2 = [] #Para testeo

for x in range(size):
    dataset1.append(pixels())
    dataset1[x].fillWithPixels()
    dataset2.append(pixels())
    dataset2[x].fillWithPixels()

    
#prueba de que anda todo bien hasta aqui:

dataset1[258][4].getData()

R: 90 G: 14 B: 52


In [48]:
#PREPARACION DEL DATASET PARA ENTRENAMIENTO
#Aqui voy a preparar el dataset para que sea compatible con la red y pueda ser usado para entrenarla:
#Seguramente tenga que convertir todo a tensores con las librerias scipy y/o numpy

#pruebo de hacerlo andar solo con los rojos (canal R):

dataset1_np = np.array(dataset1)
dataset2_np = np.array(dataset2)


#Estas cuatro asignaciones son para decirle a R-train y a r_test EL TAMAÑO QUE DEBEN TENER
#Debe haber una forma mas feliz de hacer esto pero no la descubri y si saco estas dos lineas me tira error
#------------------------------------inicio de solucion momentanea------------------------------------------
r_train = np.array(dataset1_np)
r_test = np.array(dataset1_np)
ry_train = np.zeros(10000)
ry_test = np.zeros(10000)
rx_train = np.zeros((10000,7))
rx_test = np.zeros((10000,7))
#----------------------------------------fin de solucion momentanea--------------------------------------------

k = 0

for i in range(10000):
    for j in range (8):
        r_train[i][j] = dataset1_np[i][j].R
        r_test[i][j] = dataset2_np[i][j].R
        if j == 7:
            ry_train[k] = r_train[i][j] #La salida de la red neuronal es el ULTIMO elemento de un pixels (o sea T = 7) Eso es lo que quiero predecir
            ry_test[k] = r_test[i][j]
            k = k + 1
            
for i in range(10000):
    for j in range (8):
        if j != 7:
            rx_train[i][j] = r_train[i][j]
            rx_test[i][j] = r_test[i][j]

            
rx_train = rx_train.reshape(10000,7)
rx_test = rx_test.reshape(10000,7)

#Y listo, con eso vamos a entrenar la red para UN SOLO CANAL. Mas adelante hay que hacerlo andar con los 3 canales.
#Para eso va a haber que hacer algo asi: (pero todavia no lo pude hacer andar)


#x_train = np.array(dataset1)
#y_train = x_train


#x_test = np.array(dataset2)
#y_test = x_test

In [54]:
#DEFINICION DE ARQUITECTURA DE RED NEURONAL - SEGUNDO INTENTO (el primer intento fue deleteado)

#input_dim = 256 #es el numero de entrada posible mas grande + 1, segun documentacion de keras
#model = Sequential() #Creo la red neuronal
#model.add(Embedding(input_dim, output_dim=64, input_length=7)) #Agrego la capa embedding. Suele ser la primer capa de un red en keras, siempre.

#model.add(LSTM(128)) 

#model.add(Dropout(0.5)) #Agrego capa de Dropout
#model.add(Dense(64, activation='relu'))
#model.add(Dense(1, activation='relu'))

<bound method Sequential.add of <keras.engine.sequential.Sequential object at 0x000001C9A316D2B0>>

In [103]:
#DEFINICION DE ARQUITECTURA DE RED NEURONAL - TERCER INTENTO

input_dim = 256 #es el numero de entrada posible mas grande + 1, segun documentacion de keras
model = Sequential() #Creo la red neuronal
model.add(Embedding(input_dim, output_dim=128, input_length=7)) #Agrego la capa embedding. Suele ser la primer capa de un red en keras, siempre.
model.add(LSTM(128))
# el argumento return_sequences=True en la capa LSTM, devuelve una secuencia completa
model.add(Dropout(0.5)) #Agrego capa de Dropout
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

In [104]:
#COMPILACION DE LA RED


optimizador = RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
#optimizador = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='mean_squared_error',
              optimizer=optimizador,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 7, 128)            32768     
_________________________________________________________________
lstm_18 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_26 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 33        
Total params: 185,025
Trainable params: 185,025
Non-trainable params: 0
_________________________________________________________________


In [107]:
#ENTRENAMIENTO DE LA RED
#model.fit(r_train, r_train, batch_size=1000, epochs=100)
model.fit(rx_train, ry_train, batch_size=50 ,epochs=1000)
# model.save_weights('first_try.h5') # 26% de accuracy pero mal implementada la red
# model.save_weights('second_try.h5') # 4.74% de accuracy, mejor implementada
model.save_weights('third_try.h5')
model.save('tercer_intento')
score = model.evaluate(rx_test, ry_test)
score

Epoch 1/1000
10000/10000 [==============================] - 4s 398us/step - loss: 8537.7950 - acc: 0.0039
Epoch 2/1000
10000/10000 [==============================] - 4s 379us/step - loss: 8384.9192 - acc: 0.0046
Epoch 3/1000
10000/10000 [==============================] - 4s 382us/step - loss: 8355.9656 - acc: 0.0046
Epoch 4/1000
10000/10000 [==============================] - 4s 393us/step - loss: 8260.5765 - acc: 0.0034
Epoch 5/1000
10000/10000 [==============================] - 4s 386us/step - loss: 8226.7492 - acc: 0.0042
Epoch 6/1000
10000/10000 [==============================] - 4s 376us/step - loss: 8175.1040 - acc: 0.0038
Epoch 7/1000
10000/10000 [==============================] - 4s 374us/step - loss: 8127.2077 - acc: 0.0028
Epoch 8/1000
10000/10000 [==============================] - 4s 407us/step - loss: 8037.1085 - acc: 0.0039
Epoch 9/1000
10000/10000 [==============================] - 4s 390us/step - loss: 7957.6824 - acc: 0.0038
Epoch 10/1000
10000/10000 [===================

10000/10000 [==============================] - 4s 387us/step - loss: 6097.9764 - acc: 0.0028
Epoch 76/1000
10000/10000 [==============================] - 4s 386us/step - loss: 6122.6005 - acc: 0.0040
Epoch 77/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6125.4126 - acc: 0.0037
Epoch 78/1000
10000/10000 [==============================] - 4s 397us/step - loss: 6085.9218 - acc: 0.0038
Epoch 79/1000
10000/10000 [==============================] - 4s 392us/step - loss: 6075.8712 - acc: 0.0048
Epoch 80/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6175.6905 - acc: 0.0026
Epoch 81/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6069.0300 - acc: 0.0033
Epoch 82/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6174.4357 - acc: 0.0028
Epoch 83/1000
10000/10000 [==============================] - 4s 389us/step - loss: 6120.4154 - acc: 0.0036
Epoch 84/1000
10000/10000 [========================

10000/10000 [==============================] - 4s 387us/step - loss: 6112.4671 - acc: 0.0035
Epoch 149/1000
10000/10000 [==============================] - 4s 391us/step - loss: 6106.6177 - acc: 0.0027
Epoch 150/1000
10000/10000 [==============================] - 4s 386us/step - loss: 6156.0340 - acc: 0.0036
Epoch 151/1000
10000/10000 [==============================] - 4s 386us/step - loss: 6042.5535 - acc: 0.0032
Epoch 152/1000
10000/10000 [==============================] - 4s 389us/step - loss: 6005.8317 - acc: 0.0040
Epoch 153/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6114.7558 - acc: 0.0040
Epoch 154/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6101.2990 - acc: 0.0037
Epoch 155/1000
10000/10000 [==============================] - 4s 387us/step - loss: 6062.9672 - acc: 0.0036
Epoch 156/1000
10000/10000 [==============================] - 4s 387us/step - loss: 6063.1467 - acc: 0.0043
Epoch 157/1000
10000/10000 [===============

10000/10000 [==============================] - 4s 388us/step - loss: 5971.0221 - acc: 0.0039
Epoch 224/1000
10000/10000 [==============================] - 4s 389us/step - loss: 5956.3605 - acc: 0.00390s - loss: 5946.3245 - acc
Epoch 225/1000
10000/10000 [==============================] - 4s 384us/step - loss: 6001.4032 - acc: 0.0038
Epoch 226/1000
10000/10000 [==============================] - 4s 393us/step - loss: 6008.7206 - acc: 0.00440s - loss: 6
Epoch 227/1000
10000/10000 [==============================] - 4s 385us/step - loss: 5953.8466 - acc: 0.0038
Epoch 228/1000
10000/10000 [==============================] - 4s 387us/step - loss: 5952.6270 - acc: 0.0028
Epoch 229/1000
10000/10000 [==============================] - 4s 390us/step - loss: 5984.9229 - acc: 0.0038
Epoch 230/1000
10000/10000 [==============================] - 4s 389us/step - loss: 5946.7955 - acc: 0.00370s - loss: 5942.5086 - acc: 0.
Epoch 231/1000
10000/10000 [==============================] - 4s 390us/step - loss:

10000/10000 [==============================] - 4s 394us/step - loss: 6100.2475 - acc: 0.00370s - loss: 6081.3390
Epoch 297/1000
10000/10000 [==============================] - 4s 387us/step - loss: 6064.7031 - acc: 0.0027
Epoch 298/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6116.2255 - acc: 0.0025
Epoch 299/1000
10000/10000 [==============================] - 4s 375us/step - loss: 6112.0492 - acc: 0.0030
Epoch 300/1000
10000/10000 [==============================] - 4s 376us/step - loss: 6123.3474 - acc: 0.00400s -
Epoch 301/1000
10000/10000 [==============================] - 4s 391us/step - loss: 6122.2764 - acc: 0.0027
Epoch 302/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6141.9083 - acc: 0.0040
Epoch 303/1000
10000/10000 [==============================] - 4s 389us/step - loss: 6115.9310 - acc: 0.0042
Epoch 304/1000
10000/10000 [==============================] - 4s 391us/step - loss: 6120.1725 - acc: 0.0033
Epoch 305/1000
1000

...

10000/10000 [==============================] - 4s 382us/step - loss: 6097.8214 - acc: 0.0036
Epoch 733/1000
10000/10000 [==============================] - 4s 372us/step - loss: 6070.4411 - acc: 0.0041
Epoch 734/1000
10000/10000 [==============================] - 4s 389us/step - loss: 6068.5368 - acc: 0.0032
Epoch 735/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6133.7811 - acc: 0.0027
Epoch 736/1000
10000/10000 [==============================] - 4s 395us/step - loss: 6115.1924 - acc: 0.0040
Epoch 737/1000
10000/10000 [==============================] - 4s 383us/step - loss: 6166.1347 - acc: 0.0031
Epoch 738/1000
10000/10000 [==============================] - 4s 375us/step - loss: 6128.5565 - acc: 0.0024
Epoch 739/1000
10000/10000 [==============================] - 4s 386us/step - loss: 6093.0267 - acc: 0.0037
Epoch 740/1000
10000/10000 [==============================] - 4s 387us/step - loss: 6160.0925 - acc: 0.0032
Epoch 741/1000
10000/10000 [===============

10000/10000 [==============================] - 4s 387us/step - loss: 6106.0472 - acc: 0.0038
Epoch 807/1000
10000/10000 [==============================] - 4s 387us/step - loss: 6127.5897 - acc: 0.0018
Epoch 808/1000
10000/10000 [==============================] - 4s 402us/step - loss: 6139.4760 - acc: 0.0024
Epoch 809/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6131.8839 - acc: 0.0023
Epoch 810/1000
10000/10000 [==============================] - 4s 385us/step - loss: 6088.9589 - acc: 0.0035
Epoch 811/1000
10000/10000 [==============================] - 4s 396us/step - loss: 6045.1550 - acc: 0.0027
Epoch 812/1000
10000/10000 [==============================] - 4s 389us/step - loss: 6111.3102 - acc: 0.00282s
Epoch 813/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6070.1749 - acc: 0.0027
Epoch 814/1000
10000/10000 [==============================] - 4s 391us/step - loss: 6080.1228 - acc: 0.0036
Epoch 815/1000
10000/10000 [=============

10000/10000 [==============================] - 4s 387us/step - loss: 6074.9330 - acc: 0.0025
Epoch 879/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6020.2998 - acc: 0.00320s - los
Epoch 880/1000
10000/10000 [==============================] - 4s 391us/step - loss: 5938.2804 - acc: 0.0029
Epoch 881/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6065.2566 - acc: 0.0017
Epoch 882/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6064.4864 - acc: 0.0031
Epoch 883/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6146.6558 - acc: 0.0026
Epoch 884/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6128.7747 - acc: 0.0025
Epoch 885/1000
10000/10000 [==============================] - 4s 386us/step - loss: 6027.5614 - acc: 0.0038
Epoch 886/1000
10000/10000 [==============================] - 4s 388us/step - loss: 6084.8036 - acc: 0.0038
Epoch 887/1000
10000/10000 [=======

10000/10000 [==============================] - 4s 390us/step - loss: 6188.3699 - acc: 0.0032
Epoch 953/1000
10000/10000 [==============================] - 4s 397us/step - loss: 6056.4932 - acc: 0.0029
Epoch 954/1000
10000/10000 [==============================] - 4s 390us/step - loss: 6070.7040 - acc: 0.0033
Epoch 955/1000
10000/10000 [==============================] - 4s 392us/step - loss: 6246.9282 - acc: 0.0030
Epoch 956/1000
10000/10000 [==============================] - 4s 394us/step - loss: 6218.0700 - acc: 0.0021
Epoch 957/1000
10000/10000 [==============================] - 4s 389us/step - loss: 6108.4426 - acc: 0.0037
Epoch 958/1000
10000/10000 [==============================] - 4s 392us/step - loss: 6134.0670 - acc: 0.0032
Epoch 959/1000
10000/10000 [==============================] - 4s 391us/step - loss: 6196.4167 - acc: 0.0038
Epoch 960/1000
10000/10000 [==============================] - 4s 391us/step - loss: 6179.9103 - acc: 0.0034
Epoch 961/1000
10000/10000 [===============

[9376.589590625, 0.0033]

In [66]:
Z = [0,0,0,0,0,0,0,0]
Z[0] = X[0].R
Z[1] = X[1].R
Z[2] = X[2].R
Z[3] = X[3].R
Z[4] = X[4].R
Z[5] = X[5].R
Z[6] = X[6].R
Z[7] = X[7].R
#entrada = np.array([[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]],[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]],[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]],[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]],[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]],[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]],[Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]]])
entrada = np.array([Z[0],Z[1],Z[2],Z[3],Z[4],Z[5],Z[6]])
entrada = entrada.reshape(1,7)
Y = model.predict(entrada)
Y

array([[18.56997]], dtype=float32)

In [63]:
entrada

array([[ 36, 248, 219, 110, 221, 255, 165]])

In [67]:
Z

[36, 248, 219, 110, 221, 255, 165, 99]